In [ ]:
## headings and self-function
"""
made by weiyw @ 2019-03-08
made to use the deep Spatio-temporal Residual Network
"""
# from __future__ import print_function
# from preprocessing import *
# import numpy as np
# np.random.seed(2333)  # for reproducibility


import os
import time
import struct
import numpy as np
import h5py

from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint

from STResNet import stresnet
import metrics as metrics

from PIL import Image
import matplotlib.pyplot as plt

def get_min_nonzero_value(data_in, init_min = 1):
    min_value = init_min
    for bb in data_in:
        for aa in bb:
            if abs(aa) < min_value and aa != 0:
                min_value = abs(aa)
    return min_value

def get_bindata(data_path, data_index, data_phase, nt, nr):
    # data_index start from 1
    model_num = int( ( int(data_index) - 1 ) / 51) + 1
    source_num = ( int(data_index) - 1 ) % 51 + 1
    data_name = data_path + "/model" + str(model_num) + "source" + str(source_num) + data_phase + ".bin"
    print(data_name)
    
    out_data = np.empty((nt,nr))
    FA = open(data_name, "rb")
    FA.seek(3232,0)
    for tt in range(nt):
        for rr in range(nr):
            data = FA.read(4)
            data_float = struct.unpack("f", data)[0]
            out_data[tt][rr] = data_float
     #np.shape(out_data)   # (4000,301)
#     cut_data = out_data[0:4000, 8:401]
    return out_data
    

In [ ]:
## loadin data
# data_path = "./mydata/first_test/"
data_path = "/media/wywdisk/VSPdata/data/layer2_1-90_rightPML/vz"

datapath_list = os.listdir(data_path)
datapath_list.sort()
totalNum = len(datapath_list)
print(totalNum)


# indexNum = 1
# data_name = data_path + str(indexNum) + ".png"
# print(data_name)
data_index = 1
data_phase = "vz"
nt = 4000
nr = 300
this_data = get_bindata(data_path, data_index, data_phase, nt, nr)    

In [ ]:
target = "this_data"
print "The shape of ", target, "\tis", eval(target).shape
print "The max:\t",  eval(target).max(), "\tThe min:\t",  eval(target).min()
print "The mean:\t",  eval(target).mean(), "\tThe std:\t",  eval(target).std()
print "The absolute minimal value is\t", get_min_nonzero_value(this_data)

In [ ]:
## ST-ResNet
'''
    ST-ResNet: Deep Spatio-temporal Residual Networks
    modified by weiyw @ 2019-04-01
'''

from __future__ import print_function
from keras.layers import (
    Input,
    Activation,
    merge,
    Dense,
    Reshape
)
from keras.layers.convolutional import Convolution2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
#from keras.utils.visualize_util import plot


def _shortcut(input, residual):
    return merge([input, residual], mode='sum')


def _bn_relu_conv(nb_filter, nb_row, nb_col, subsample=(1, 1), bn=False):
    def f(input):
        if bn:
            input = BatchNormalization(mode=0, axis=1)(input) ## attention!!
        activation = Activation('relu')(input)
        return Convolution2D(nb_filter=nb_filter, nb_row=nb_row, nb_col=nb_col, subsample=subsample, border_mode="same")(activation)
    return f


def _residual_unit(nb_filter, init_subsample=(1, 1)):
    def f(input):
        residual = _bn_relu_conv(nb_filter, 3, 3)(input)
        residual = _bn_relu_conv(nb_filter, 3, 3)(residual)
        return _shortcut(input, residual)
    return f


def ResUnits(residual_unit, nb_filter, repetations=1):
    def f(input):
        for i in range(repetations):
            init_subsample = (1, 1)
            input = residual_unit(nb_filter=nb_filter,
                                  init_subsample=init_subsample)(input)
        return input
    return f


# def stresnet(c_conf=(3, 2, 32, 32), p_conf=(3, 2, 32, 32), t_conf=(3, 2, 32, 32), external_dim=8, nb_residual_unit=3):
def stresnet(total_conf=(3, 2, 32, 32), external_dim=None, nb_residual_unit=3):
    '''
    C - Temporal Closeness
    P - Period
    T - Trend
    conf = (len_seq, nb_flow, map_height, map_width)
    external_dim
    '''
    # main input
    main_inputs = []
    outputs = []
    for conf in [total_conf] :#[c_conf, p_conf, t_conf]:
        if conf is not None:
            len_seq, nb_flow, map_height, map_width = conf
            input = Input(shape=(nb_flow * len_seq, map_height, map_width))
            main_inputs.append(input)
            # Conv1
            conv1 = Convolution2D(
                nb_filter=64, nb_row=2, nb_col=2, border_mode="same")(input)
#             conv1 = Convolution2D(
#                 nb_filter=64, nb_row=3, nb_col=3, border_mode="same")(input)
            # [nb_residual_unit] Residual Units
            residual_output = ResUnits(_residual_unit, nb_filter=64,
                              repetations=nb_residual_unit)(conv1)
            # Conv2
            activation = Activation('relu')(residual_output)
#             conv2 = Convolution2D(
#                 nb_filter=nb_flow, nb_row=3, nb_col=3, border_mode="same")(activation)
            conv2 = Convolution2D(
                nb_filter=nb_flow, nb_row=2, nb_col=2, border_mode="same")(activation)

            outputs.append(conv2)

    # parameter-matrix-based fusion
    if len(outputs) == 1:
        main_output = outputs[0]
    else:
        from .iLayer import iLayer
        new_outputs = []
        for output in outputs:
            new_outputs.append(iLayer()(output))
        main_output = merge(new_outputs, mode='sum')

    # fusing with external component
    if external_dim != None and external_dim > 0:
        # external input
        external_input = Input(shape=(external_dim,))
        main_inputs.append(external_input)
        embedding = Dense(output_dim=10)(external_input)
        embedding = Activation('relu')(embedding)
        h1 = Dense(output_dim=nb_flow * map_height * map_width)(embedding)
        activation = Activation('relu')(h1)
        external_output = Reshape((nb_flow, map_height, map_width))(activation)
        main_output = merge([main_output, external_output], mode='sum')
    else:
        print('external_dim:', external_dim)

    main_output = Activation('tanh')(main_output)
    model = Model(input=main_inputs, output=main_output)

    return model

# if __name__ == '__main__':
# #     model = stresnet(external_dim=28, nb_residual_unit=12)
#     model = stresnet(external_dim=None, nb_residual_unit=8)
#     #plot(model, to_file='ST-ResNet.png', show_shapes=True)
#     model.summary()


In [ ]:
## evaluation
# import numpy as np
from keras import backend as K
def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))


def root_mean_square_error(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5


def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

# aliases
mse = MSE = mean_squared_error
rmse = RMSE = root_mean_square_error

In [ ]:
model = stresnet(external_dim=None, nb_residual_unit=8)
model.summary()

In [ ]:
adam = Adam(lr=lr)
model.compile(loss='mse', optimizer=adam, metrics=[rmse])
 #   model.summary()